# U-Net Model
In this notebook I examine the performance of the U-NEet model. 

In [ ]:
import sys
sys.path.insert(0, "..")
import gc
gc.enable()

In [ ]:
from utils.DataLoader import DataLoader
from utils.utilities import *
from utils.visualization import *
from models.unet import UNet
import tensorflow as tf

print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

tf.debugging.set_log_device_placement(True)

### Loading the data and splitting them into training and validation sets

In [ ]:
dl = DataLoader()
train_df, valid_df = dl.train_split(0.05, filterNan=True)
print("train set shape", train_df.shape)
print("validation set shape", valid_df.shape)

### The images are Loaded through a generator which is forwarded to a data augmentation generator

In [ ]:
grouped_images_gen = dl.get_grouped_images_gen(train_df, f="sobel_op") # applying filters increases dramatically the execution time
aug_gen = get_augmented_images_generator(grouped_images_gen)
valid_x, valid_y = next(dl.get_grouped_images_gen(valid_df, batch_size=1000, f="sobel_op")) #valid_df.shape[0]))

### An example

In [ ]:
train_x, train_y = next(aug_gen)
plot_masks(train_x[6], train_y[6])

## Initializing U-Net Model

In [ ]:
t_x, t_y = next(aug_gen)
x_shape =np.squeeze(t_x[0]).shape
unet_model = UNet(x_shape, "../models/serialized/")
unet_model.compile()

## Training

In [ ]:
hl = unet_model.validate(aug_gen, train_df.shape[0], valid_set=(valid_x, valid_y), epochs=20, train_steps=500)
unet_model.show_loss(hl)

## Examine its Performance in Testing set.
Even though the relatively low IoU, its perdormance is quite good resulting to good segmentation masks. Using filters
such as the _equalizer_ and the _adaptive equalizer_ improve its performance.

In [ ]:
test_x, test_y = next(dl.get_grouped_images_gen(batch_size=50, train=False, f="sobel_op"))
unet_model.examine_performance(test_x, test_y, load=True)
